## Analyzing a scene

We are going to apply the different capabilities build in order to predict actors in a scene

### Importing libraries

In [ ]:
import sys
import os
import json
import pandas as pd
import numpy as np
!pip install pandasql
from pandasql import sqldf
sys.path.append('/opt/workspace/src/python_scripts/')
from ops_face_recognition import train_recognizer,get_probabilities_for_folder
from ops_files_operations import read_pickle_file, read_json_file, get_element_from_metadata
from ops_face_detection import process_video, get_video_embeddings,get_frame_from_video, get_embeddings_from_image

### Creating best recognizers

Bases on the results obtained in Notebook "10_training_models"

In [ ]:
embeddings_folder = './models/embeddings/actor_faces'
recognizer_path = './models/recognizers'

#### Linear recognizer

In [ ]:
train_recognizer(
    embeddings_folder=embeddings_folder,
    model_id = None,
    C=1.0,
    kernel='linear',
    probability=True,
    save_to_pickle=True,
    output_folder=recognizer_path
)

#### RBF recognizer

In [ ]:
train_recognizer(
    embeddings_folder=embeddings_folder,
    model_id = None,
    kernel='rbf',
    C=1.0,
    gamma=1.0,
    probability=True,
    save_to_pickle=True,
    output_folder=recognizer_path
)

#### Poly recognizer

In [ ]:
train_recognizer(
    embeddings_folder=embeddings_folder,
    model_id = None,
    kernel='poly',
    C=1.0,
    degree=3,
    probability=True,
    save_to_pickle=True,
    output_folder=recognizer_path
)

#### Getting recognizers ids

In [ ]:
recognizers_metadata_path = './models/recognizers/recognizer_metadata.json'
linear_recognizer_id = get_element_from_metadata(
    metadata_file_path=recognizers_metadata_path,
    key='kernel',
    value='linear',
    latest=True
)["recognizer_id"]
rbf_recognizer_id = get_element_from_metadata(
    metadata_file_path=recognizers_metadata_path,
    key='kernel',
    value='rbf',
    latest=True
)["recognizer_id"]
poly_recognizer_id = get_element_from_metadata(
    metadata_file_path=recognizers_metadata_path,
    key='kernel',
    value='poly',
    latest=True
)["recognizer_id"]
print(f'Linear = {linear_recognizer_id}, rbf = {rbf_recognizer_id}, poly = {poly_recognizer_id}.')

### Processing a scene

#### Obtaining scene embeddings

In [ ]:
scene_path = './datasets/videos/the_final_kick.mp4'
output_path = './models/embeddings/processed_videos'
get_video_embeddings(
    video_path = scene_path,
    results_path = output_path,
    partitions=4, 
    desired_fps=4
)

#### Getting embeddings "processed_video_id"

In [ ]:
probs_metadata_path = './models/embeddings/processed_videos/the_final_kick/processed_videos_metadata.json'
processed_video_id = get_element_from_metadata(
    metadata_file_path=probs_metadata_path,
    latest=True
)["processed_video_id"]
print(f'Processed video id = {processed_video_id}.')

#### Predicting results for that scene

In [ ]:
scene_embeddings_path = './models/embeddings/processed_videos/the_final_kick'
results_folder = './models/results'
folder_probabilities, folder_probabilities_metadata = get_probabilities_for_folder(
    folder_path = scene_embeddings_path,
    recognizer_folder = recognizer_path,
    save_to_pickle = True,
    output_folder = results_folder,
    processed_video_id = processed_video_id,
    recognizer_id = linear_recognizer_id
)

In [ ]:
results_id = folder_probabilities_metadata['results_id']
results_id

### Understanding results

#### Getting results dict

In [2]:
results_metadata_path = './models/results/probabilities_metadata.json'
results_metadata = get_element_from_metadata(
    metadata_file_path=results_metadata_path,
    #key = 'results_id',
    #value = results_id,
    latest=True
)
pickle_path = results_metadata['pickle_path']
results = read_pickle_file(pickle_path)

In [9]:
results[0]["probabilities"][15]

{'frame_number': '474',
 'timestamp': '00:00:19.770',
 'predictions': [{'2037_cillian_murphy': 0.5326179425375231,
   '6193_leonardo_dicaprio': 0.1221048560387834,
   '2524_tom_hardy': 0.06128185310235346,
   '526_lukas_haas': 0.055021633167948704,
   '3899_ken_watanabe': 0.05138065273911555,
   '13022_tom_berenger': 0.049764110754460696,
   '4935_pete_postlethwaite': 0.035297353536172714,
   '95697_dileep_rao': 0.028151471865901546,
   '3895_michael_caine': 0.023959367287635286,
   '8293_marion_cotillard': 0.015659827217873526,
   '27578_elliot_page': 0.014380513071059482,
   '24045_joseph_gordon-levitt': 0.010380418681172843}]}

#### Transforming dict into pd dataframe

In [ ]:
full_probs = []
for chunk in results:
    for preds in chunk["probabilities"]:
        preds_dict= {}
        preds_dict["frame_number"] = preds["frame_number"]
        preds_dict["timestamp"] = preds["timestamp"]
        preds_dict["pred_1_name"] = sorted(preds["predictions"][0], key=preds["predictions"][0].get, reverse=True)[0]
        preds_dict["pred_1_value"] = preds["predictions"][0][preds_dict["pred_1_name"]]
        preds_dict["pred_2_name"] = sorted(preds["predictions"][0], key=preds["predictions"][0].get, reverse=True)[1]
        preds_dict["pred_2_value"] = preds["predictions"][0][preds_dict["pred_2_name"]]
        preds_dict["pred_3_name"] = sorted(preds["predictions"][0], key=preds["predictions"][0].get, reverse=True)[2]
        preds_dict["pred_3_value"] = preds["predictions"][0][preds_dict["pred_3_name"]]
        full_probs.append(preds_dict)

results_df = pd.DataFrame(full_probs)

In [ ]:
sqldf("""
    select
        *
    from results_df
    where 
        pred_1_name like '%murphy%'
""")

In [ ]:
scene_path = './datasets/videos/the_final_kick.mp4'
frame = get_frame_from_video(video_path = scene_path,timestamp = '00:01:08.000')
embs, embs_metadata = get_embeddings_from_image(provided_image=frame, multiple_faces=True, display=True)